In [10]:
import pandas as pd

In [1]:
import pickle as pkl

In [6]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [6]:
with open('features_juli_with_support.pkl', 'rb') as file:
    train = pkl.load(file)
    

In [53]:
with open('test_features_juli_with_support.pkl', 'rb') as file:
    test = pkl.load(file)

# Con mis features

In [7]:
train.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'label', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [8]:
train.shape

(19414, 13)

In [9]:
X_train = train[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia','modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [10]:
y_train = train.label

In [19]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(X_train,y_train,test_size=0.2)

In [20]:
model = ExtraTreesClassifier(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)



In [25]:
model.fit(Xtrn,Ytrn)
pred= model.predict_proba(Xtest)


In [26]:
pred.shape

(3883, 2)

In [27]:
Ytest.shape

(3883,)

In [30]:
roc_auc_score(Ytest, pred[:,1])

0.6358333664902718

In [31]:
help(ExtraTreesClassifier)

Help on class ExtraTreesClassifier in module sklearn.ensemble.forest:

class ExtraTreesClassifier(ForestClassifier)
 |  An extra-trees classifier.
 |  
 |  This class implements a meta estimator that fits a number of
 |  randomized decision trees (a.k.a. extra-trees) on various sub-samples
 |  of the dataset and uses averaging to improve the predictive accuracy
 |  and control over-fitting.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |      .. versionchanged:: 0.20
 |         The default value of ``n_estimators`` will change from 10 in
 |         version 0.20 to 100 in version 0.22.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  max_depth : integer or None, optional (default

In [50]:
from sklearn.model_selection import GridSearchCV

In [48]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid.fit(Xtrn, Ytrn)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [49]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [50]:
grid.best_score_

0.9499710256905544

In [52]:
roc_auc_score(Ytest, grid.predict_proba(Xtest)[:,1])

0.6685247814957758

In [55]:
test.columns

Index(['visited site', 'generic listing', 'conversion', 'viewed product',
       'person', 'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support'],
      dtype='object')

In [56]:
x_test = test[['visited site', 'generic listing', 'conversion', 'viewed product',
        'tiempo_promedio_por_dia', 'modelo_mas_visto',
       'condition_mode', 'storage_mode', 'campaing_mode', 'cant_model_vistos',
       'event_support']]

In [75]:
sum(pd.isnull(x_test['event_support']))

0

In [66]:
x_test['viewed product'].fillna(x_test['viewed product'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [60]:
x_test['visited site'].fillna(x_test['visited site'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [62]:
x_test['generic listing'].fillna(x_test['generic listing'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [64]:
x_test['conversion'].fillna(x_test['conversion'].mean(), inplace = True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [79]:
pred_porta = grid.predict_proba(x_test)[:,1]

In [77]:
to_puvlic = pd.DataFrame()

In [78]:
to_puvlic['person'] = test.person

In [80]:
to_puvlic['label'] = pred_porta

In [82]:
to_puvlic.to_csv('extraTreesJuli.csv', index = False)

# Con features de las chicas

In [86]:
with open('features_chicas.pkl','rb') as file:
    features = pkl.load(file)

In [85]:
features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [87]:
features['device_frecuente'].fillna(features['device_frecuente'].mode()[0], inplace=True)

In [88]:
features[ 'Returning'].fillna(features['Returning'].mode()[0], inplace=True)

In [89]:
import category_encoders as ce

In [90]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [91]:
label_train = pd.read_csv('data/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [92]:
train_features = pd.merge(features, label_train, on='person', how='right')

In [93]:
train_features.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'label'],
      dtype='object')

In [94]:
sum(pd.isnull(train_features['device_frecuente']))

0

In [95]:
train_features['Returning'].fillna(train_features['Returning'].mode()[0], inplace=True)

In [96]:
train_features['device_frecuente'].fillna(train_features['device_frecuente'].mode()[0], inplace=True)

In [97]:
x_train_features = train_features[[ 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [98]:
y_train_features = train_features.label

In [99]:
x_train_features.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(x_train_features.channel_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [100]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [101]:
x_train_features.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(x_train_features.device_frecuente), y_train_features)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [102]:
x_train_features.shape

(19414, 27)

In [103]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn2, Xtest2, Ytrn2, Ytest2 = train_test_split(x_train_features,y_train_features,test_size=0.2)

In [104]:
param_grid2C = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid2C = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid2C.fit(Xtrn2, Ytrn2)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.9min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [107]:
grid2C.best_score_

0.9484257291867877

In [108]:
pred = grid2C.predict_proba(Xtest2)[:,1]

In [109]:
roc_auc_score(Ytest2,pred)

0.8181185461492971

In [110]:
test_person = pd.read_csv('../7506_ml/data/trocafone_kaggle_test.csv')

In [111]:
test_person.shape

(19415, 1)

In [112]:
test2 = pd.merge(features , test_person, on='person', how='right')

In [113]:
test2.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(test2.channel_frecuente))

In [114]:
test2.device_frecuente = te_device_frecuente.transform(pd.DataFrame(test2.device_frecuente))

In [115]:
test2.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

In [116]:
xtest2 = test2[['cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10']]

In [117]:
sum(pd.isnull(xtest2['device_frecuente']))

0

In [118]:
xtest2['channel_frecuente'].fillna(xtest2['channel_frecuente'].mode()[0], inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [120]:
pred = grid2C.predict_proba(xtest2)[:,1]

In [121]:
to_public = pd.DataFrame()

In [122]:
to_public['person'] = test2.person

In [123]:
to_public['label'] = pred

In [124]:
to_public.to_csv('extraTrees5.csv', index=False)

In [125]:
grid2C.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 100}

In [126]:
model2C = ExtraTreesClassifier(bootstrap=True, max_features='auto', min_samples_split=5, n_estimators=100)

In [127]:
model2C.fit(Xtrn2, Ytrn2)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [128]:
pred2C = model2C.predict_proba(Xtest2)

In [129]:
roc_auc_score(Ytest2, pred2C[:,1])

0.812842499507195

In [130]:
model2C.feature_importances_

array([0.0650609 , 0.05033752, 0.06440783, 0.11130231, 0.04042169,
       0.0560851 , 0.00877165, 0.05208616, 0.05447614, 0.02823737,
       0.06373246, 0.04027697, 0.03736303, 0.07285524, 0.04791493,
       0.04260922, 0.03184851, 0.00770149, 0.00877217, 0.00852902,
       0.00952757, 0.01270548, 0.01349369, 0.01497945, 0.01542966,
       0.01897062, 0.02210382])

In [131]:
prod_cols = list(Xtrn2.columns)
d ={}
for p in prod_cols:
    y = np.array(Xtrn2[p])
    l = zip(Xtrn2.columns, model2C.feature_importances_)
    d[p] = sorted(l, key=lambda x: x[1], reverse=True)

In [132]:
d

{'Returning': [('cant_ch', 0.11130230923184908),
  ('tiempo_total', 0.0728552424425288),
  ('cant_eventos_totales', 0.06506090029705196),
  ('cant_bl', 0.06440782781121908),
  ('cant_vp', 0.06373245730849121),
  ('cant_gl', 0.05608509836264201),
  ('cant_s_p', 0.054476136067570693),
  ('cant_seh', 0.052086157699442165),
  ('cant_ach', 0.05033752241965469),
  ('tiempo_new_returning', 0.047914934424758936),
  ('channel_frecuente', 0.04260921827718438),
  ('cant_cv', 0.0404216890429879),
  ('cant_vs', 0.04027697036531838),
  ('Returning', 0.03736303457272459),
  ('device_frecuente', 0.031848509022074076),
  ('cant_sp', 0.02823737330114091),
  ('q10', 0.02210382401028583),
  ('q9', 0.018970617847955125),
  ('q8', 0.015429655752526557),
  ('q7', 0.014979452861978753),
  ('q6', 0.013493685298419994),
  ('q5', 0.012705476912496787),
  ('q4', 0.009527572838711147),
  ('q2', 0.00877216884253286),
  ('cant_l', 0.008771650397294068),
  ('q3', 0.008529021295755604),
  ('q1', 0.0077014932954044095)

In [133]:
param_grid3C = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid3C = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid3C.fit(Xtrn2[['cant_ch',
  'tiempo_total',
  'cant_eventos_totales',
  'cant_bl',
  'cant_vp',
  'cant_gl',
  'cant_s_p',
  'cant_seh',
  'cant_ach']], Ytrn2)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [139]:
grid3C.best_score_

0.9482325671238169

In [136]:
pred = grid3C.predict_proba(Xtest2[['cant_ch',
  'tiempo_total',
  'cant_eventos_totales',
  'cant_bl',
  'cant_vp',
  'cant_gl',
  'cant_s_p',
  'cant_seh',
  'cant_ach']])

In [138]:
roc_auc_score(Ytest2, pred[:,1])

0.7838238638948278

In [140]:
param_grid4C = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid4C = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=1)

grid4C.fit(Xtrn2[['cant_ch',
  'tiempo_total',
  'cant_eventos_totales',
  ]], Ytrn2)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  2.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [142]:
pred = grid4C.predict_proba(Xtest2[['cant_ch',
  'tiempo_total',
  'cant_eventos_totales',
  ]])

In [143]:
roc_auc_score(Ytest2, pred[:,1])

0.7635854978847291

# Con el de Mati

In [2]:
df = pkl.load(open('Forma1/feature.pkl','rb'))

In [3]:
df.columns

Index(['person', 'label', 'storage_mode', 'event_ad campaign hit',
       'event_brand listing', 'event_checkout', 'event_conversion',
       'event_generic listing', 'event_lead', 'event_search engine hit',
       'event_searched products', 'event_staticpage', 'event_viewed product',
       'event_visited site', 'new_vs_returning_New',
       'new_vs_returning_Returning', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'condition_Novo', 'time_cos_by_day'],
      dtype='object')

In [4]:
xTrain = df[['storage_mode', 'event_ad campaign hit',
       'event_brand listing', 'event_checkout', 'event_conversion',
       'event_generic listing', 'event_lead', 'event_search engine hit',
       'event_searched products', 'event_staticpage', 'event_viewed product',
       'event_visited site', 'new_vs_returning_New',
       'new_vs_returning_Returning', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'condition_Novo', 'time_cos_by_day']]

In [40]:
sum(pd.isnull(xTrain['time_cos_by_day']))

0

In [39]:
xTrain['condition_Novo'].fillna(xTrain['condition_Novo'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [36]:
xTrain['condition_Muito Bom'].fillna(xTrain['condition_Muito Bom'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [34]:
xTrain['condition_Excelente'].fillna(xTrain['condition_Excelente'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [26]:
xTrain['new_vs_returning_New'].fillna(xTrain['new_vs_returning_New'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [28]:
xTrain['new_vs_returning_Returning'].fillna(xTrain['new_vs_returning_Returning'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [32]:
xTrain['condition_Bom - Sem Touch ID'].fillna(xTrain['condition_Bom - Sem Touch ID'].mode()[0],inplace=True)

/home/july/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [ ]:
xTrain['condition_Bom'].fillna(xTrain['condition_Bom'].mode()[0],inplace=True)

In [5]:
yTrain = df.label

In [41]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn3, Xtest3, Ytrn3, Ytest3 = train_test_split(xTrain,yTrain,test_size=0.2)

In [42]:
model = ExtraTreesClassifier(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)

In [43]:
model.fit(Xtrn3,Ytrn3)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [47]:
prde = model.predict_proba(Xtest3)[:,1]

In [48]:
roc_auc_score(Ytest3, prde)

0.6560473798533805

In [51]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=10)

grid.fit(Xtrn3, Ytrn3)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  4.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [52]:
grid.best_score_

0.9495203142102892

In [53]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [55]:
pred = grid.predict_proba(Xtest3)

In [57]:
roc_auc_score(Ytest3, pred[:,1])

0.7670194135215856

In [58]:
model = ExtraTreesClassifier(n_estimators=150, max_features='auto', min_samples_leaf=1, min_samples_split=5, bootstrap=True) 

In [59]:
model.fit(Xtrn3,Ytrn3)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [62]:
model.feature_importances_

array([0.02459922, 0.05831588, 0.06302309, 0.09889965, 0.04121445,
       0.06125702, 0.01549358, 0.06063746, 0.05305341, 0.02650696,
       0.0621846 , 0.06489308, 0.04257161, 0.04275541, 0.05331776,
       0.03730013, 0.04943023, 0.05123995, 0.0161005 , 0.07720601])

In [63]:
Xtrn3.columns

Index(['storage_mode', 'event_ad campaign hit', 'event_brand listing',
       'event_checkout', 'event_conversion', 'event_generic listing',
       'event_lead', 'event_search engine hit', 'event_searched products',
       'event_staticpage', 'event_viewed product', 'event_visited site',
       'new_vs_returning_New', 'new_vs_returning_Returning', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'condition_Novo', 'time_cos_by_day'],
      dtype='object')

In [65]:
import numpy as np

In [67]:
prod_cols = list(Xtrn3.columns)
d ={}
for p in prod_cols:
    y = np.array(Xtrn3[p])
    l = zip(Xtrn3.columns, model.feature_importances_)
    d[p] = sorted(l, key=lambda x: x[1], reverse=True)


In [68]:
d

{'condition_Bom': [('event_checkout', 0.09889964844713243),
  ('time_cos_by_day', 0.07720601233887307),
  ('event_visited site', 0.06489308294072184),
  ('event_brand listing', 0.06302309418033802),
  ('event_viewed product', 0.06218460317460772),
  ('event_generic listing', 0.06125701597876419),
  ('event_search engine hit', 0.06063746069568837),
  ('event_ad campaign hit', 0.058315875723694774),
  ('condition_Bom', 0.05331775657750673),
  ('event_searched products', 0.05305340816489245),
  ('condition_Muito Bom', 0.05123994825202425),
  ('condition_Excelente', 0.04943022519818199),
  ('new_vs_returning_Returning', 0.04275541270652264),
  ('new_vs_returning_New', 0.042571614229455185),
  ('event_conversion', 0.04121445220341182),
  ('condition_Bom - Sem Touch ID', 0.037300129128193436),
  ('event_staticpage', 0.026506956196759534),
  ('storage_mode', 0.024599222177712158),
  ('condition_Novo', 0.016100499219363257),
  ('event_lead', 0.015493582466156)],
 'condition_Bom - Sem Touch ID'

In [60]:
pred = model.predict_proba(Xtest3)

In [61]:
roc_auc_score(Ytest3, pred[:,1])

0.7632113765951669

In [69]:
model.fit(Xtrn3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  'event_brand listing',
  'event_viewed product',
  'event_generic listing', 
  'event_search engine hit',
  'event_ad campaign hit',
  'condition_Bom', 
  'event_searched products', 
  'condition_Muito Bom']], Ytrn3)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [71]:
pred = model.predict_proba(Xtest3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  'event_brand listing',
  'event_viewed product',
  'event_generic listing', 
  'event_search engine hit',
  'event_ad campaign hit',
  'condition_Bom', 
  'event_searched products', 
  'condition_Muito Bom']])

In [73]:
roc_auc_score(Ytest3, pred[:,1])

0.7597278034211241

In [74]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=10)

grid.fit(Xtrn3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  'event_brand listing',
  'event_viewed product',
  'event_generic listing', 
  'event_search engine hit',
  'event_ad campaign hit',
  'condition_Bom', 
  'event_searched products', 
  'condition_Muito Bom']], Ytrn3)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [75]:
grid.best_score_

0.9496490889189363

In [76]:
pred = grid.predict_proba(Xtest3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  'event_brand listing',
  'event_viewed product',
  'event_generic listing', 
  'event_search engine hit',
  'event_ad campaign hit',
  'condition_Bom', 
  'event_searched products', 
  'condition_Muito Bom']])

In [78]:
roc_auc_score(Ytest3, pred[:,1])

0.7660175128970947

In [79]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 100}

In [81]:
param_grid = { 
          "n_estimators"      : [50,100,150],
          "max_features"      : ["auto"],
          "min_samples_split" : [2,3,4,5] ,
          "bootstrap": [True]
          }
grid2 = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=10)

grid2.fit(Xtrn3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  ]], Ytrn3)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  2.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'max_features': ['auto'], 'min_samples_split': [2, 3, 4, 5], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [82]:
grid2.best_score_

0.9495203142102892

In [84]:
pred = grid2.predict_proba(Xtest3[['event_checkout',
  'time_cos_by_day',
    'event_visited site',
  ]])

In [85]:
roc_auc_score(Ytest3, pred[:,1])

0.735109285908227

# Features mica mas Mati